In [36]:
import pandas as pd
import requests
import numpy as np
from bs4 import BeautifulSoup, Comment
import re
import time
import pandasql as psql

In [12]:
# functions
def get_drive_table(team, soup):
    if team == 'home':
        ids = ['all_home_drives', 'home_drives']
    if team == 'vis':
        ids = ['all_vis_drives', 'vis_drives']
    drive_data = []
    parent = soup.find('div', {'id': ids[0]})
    team = parent.find('h2').get_text(strip=False)
    team = team.split()
    team = team[0]
    print(team)
    if parent:
        # print('found parent')
        div = parent.find(string=lambda text: isinstance(text, Comment))
        if div:
            comment_soup = BeautifulSoup(str(div), 'html.parser')
            table = comment_soup.find('table', {'id': ids[1]})
            if table:
                # print('found table')
                cols = table.find('thead').find('tr')
                column_headers = [th.get_text(strip=True) for th in cols.find_all('th')]
                drives = table.find('tbody').find_all('tr')
                for drive in drives:
                    columns = drive.find_all(['th', 'td'])
                    row_data = [column.get_text(strip=False) for column in columns]
                    drive_data.append(row_data)
    drives_df = pd.DataFrame(columns = column_headers, data = drive_data)
    drives_df['team'] = team
    return drives_df


def scrape_pbp(game_page_soup):
    pbp_datas = []
    pbp_parent = game_page_soup.find('div', {'class': 'table_wrapper', 'id': 'all_pbp'})
    if pbp_parent:
        pbp_div = pbp_parent.find(string=lambda text: isinstance(text, Comment))
        if pbp_div:
            comment_soup = BeautifulSoup(str(pbp_div), 'html.parser')
            pbp_table = comment_soup.find('table', {'id': 'pbp'})
            if pbp_table:
                cols = pbp_table.find('thead').find('tr')
                column_headers = [th.get_text(strip=True) for th in cols.find_all('th')]
                plays = pbp_table.find('tbody').find_all('tr',{'class': ''})
                for play in plays:
                    columns = play.find_all(['th', 'td'])
                    play_data = [column.get_text(strip=False) for column in columns]
                    pbp_datas.append(play_data)

    pbp_datas = pd.DataFrame(columns = column_headers, data = pbp_datas)
    return pbp_datas

In [9]:
# scraping 1 game
pbp_data = []

game_url = 'https://www.pro-football-reference.com/boxscores/202309070kan.htm'
r = requests.get(game_url)
print(f'status: {r.status_code}')

game_page_soup = BeautifulSoup(r.text)

status: 200


In [53]:
# scraping drive data for home and away team
home_drives = get_drive_table('home', game_page_soup)
vis_drives = get_drive_table('vis', game_page_soup)

# getting home and away team variables
home_team = home_drives['team'][0]
vis_team = vis_drives['team'][0]

drives = pd.concat([home_drives, vis_drives], axis=0)
drives['Quarter'] = drives['Quarter'].astype(int)
drives['minute'] = drives['Time'].str.extract(r'([0-9]+):').astype(int)
drives['seconds'] = drives['Time'].str.extract(r'[0-9]+:([0-9]+)').astype(int)
drives['seconds_ratio'] = (drives['seconds'] / 60).astype(float)
drives['Numeric_time'] = drives['minute'] + drives['seconds_ratio']
drives = drives.sort_values(by=['Quarter', 'Numeric_time'], ascending=[True, False]).reset_index()
# drives['end_time'] = drives['Numeric_time'].shift(-1)
# drives['end_quarter'] = drives['Quarter'].shift(-1)
# drives['end_time'] = drives['end_time'].fillna(0)
# drives['end_quarter'] = drives['end_quarter'].fillna(4)
drives = drives.drop(columns=['minute', 'seconds', 'seconds_ratio', 'index', '#'])

Chiefs
Lions


In [54]:
drives

,Quarter,Time,LOS,Plays,Length,Net Yds,Result,team,Numeric_time,end_time,end_quarter
0,1,15:00,DET 25,3,0:51,7,Punt,Lions,15.000000,14.150000,1.0
1,1,14:09,KAN 13,6,3:19,25,Punt,Chiefs,14.150000,10.833333,1.0
2,1,10:50,DET 9,14,8:04,91,Touchdown,Lions,10.833333,2.766667,1.0
3,1,2:46,KAN 25,13,5:56,75,Touchdown,Chiefs,2.766667,11.833333,2.0
4,2,11:50,DET 25,11,6:36,61,Fumble,Lions,11.833333,5.233333,2.0
5,2,5:14,KAN 7,3,1:27,5,Punt,Chiefs,5.233333,3.783333,2.0
6,2,3:47,DET 41,3,1:45,2,Punt,Lions,3.783333,2.033333,2.0
7,2,2:02,KAN 18,6,1:28,82,Touchdown,Chiefs,2.033333,0.566667,2.0
8,2,0:34,DET 25,7,0:34,17,Downs,Lions,0.566667,15.000000,3.0
9,3,15:00,KAN 25,5,2:06,14,Punt,Chiefs,15.000000,12.900000,3.0


In [13]:
# scraping pbp data
pbp_data = scrape_pbp(game_page_soup)

# Setting up receiving and kicking teams
coin_toss = pbp_data.iloc[0]
pbp_data = pbp_data.drop(0)
coin_toss = coin_toss[7]
teams = re.findall(r'\b[A-Z][a-zA-Z]*\b', coin_toss)
match = re.search(r"(\w+)\s+to\s+receive\s+the\s+opening\s+kickoff", coin_toss)
if match:
    receiving_team = match.group(1)

# dropping timeouts
pbp_data = pbp_data[pbp_data['Location'].str.strip() != '']
pbp_data = pbp_data.dropna(subset=['Location'])

print(f'{home_team} vs. {vis_team} \n total plays: {len(pbp_data)}')

Chiefs vs. Lions 
 total plays: 132


In [42]:
# General Cleaning
pbp_data['Quarter'] = pbp_data['Quarter'].astype(int)
pbp_data['field_side'] = pbp_data['Location'].str.extract(r'([A-Z]+)')
pbp_data['yardline'] = pbp_data['Location'].str.extract(r'([0-9]+)')
pbp_data['yardline'] = pbp_data['yardline'].astype(int)
pbp_data['minute'] = pbp_data['Time'].str.extract(r'([0-9]+):').astype(int)
pbp_data['seconds'] = pbp_data['Time'].str.extract(r'[0-9]+:([0-9]+)').astype(int)
pbp_data['seconds_ratio'] = (pbp_data['seconds'] / 60).astype(float)
pbp_data['Numeric_time'] = pbp_data['minute'] + pbp_data['seconds_ratio']
pbp_data = pbp_data.drop(columns=['minute', 'seconds', 'seconds_ratio'])

# what I need to do is change my numeric time to start at 0 and go to 60 so that it encompasses all 4 quarters.  Do this for
# drives and pbp and then you can just join when its between those and disregard quarter.
pysqldf = lambda q: psql.sqldf(q, globals())
query = f"""
        SELECT * 
        FROM pbp_data 
        LEFT JOIN drives on (drives.Quarter = pbp_data.Quarter OR drives.end_quarter = pbp_data.Quarter) AND 
        drives.Numeric_time <= pbp_data.Numeric_time 
        ORDER BY Quarter, Numeric_time DESC
        """
merged = pysqldf(query)
# pbp_data.merge((drives['Quarter'] == pbp_data['Quarter']) & (drives['Numeric_time'] <= pbp_data['Numeric_time']), how='inner')




In [23]:
drives

,Quarter,Time,LOS,Plays,Length,Net Yds,Result,team,Numeric_time
0,1,15:00,DET 25,3,0:51,7,Punt,Lions,15.000000
1,1,14:09,KAN 13,6,3:19,25,Punt,Chiefs,14.150000
2,1,10:50,DET 9,14,8:04,91,Touchdown,Lions,10.833333
3,1,2:46,KAN 25,13,5:56,75,Touchdown,Chiefs,2.766667
4,2,11:50,DET 25,11,6:36,61,Fumble,Lions,11.833333
5,2,5:14,KAN 7,3,1:27,5,Punt,Chiefs,5.233333
6,2,3:47,DET 41,3,1:45,2,Punt,Lions,3.783333
7,2,2:02,KAN 18,6,1:28,82,Touchdown,Chiefs,2.033333
8,2,0:34,DET 25,7,0:34,17,Downs,Lions,0.566667
9,3,15:00,KAN 25,5,2:06,14,Punt,Chiefs,15.000000


In [41]:
pbp_data

,Quarter,Time,Down,ToGo,Location,DET,KAN,Detail,EPB,EPA,field_side,yardline,Numeric_time
0,1,15:00,,,KAN 35,0,0,"Harrison Butker kicks off 65 yards, touchback.",0.000,0.610,KAN,35,15.000000
1,1,15:00,1,10,DET 25,0,0,David Montgomery up the middle for 7 yards (ta...,0.610,1.010,DET,25,15.000000
2,1,14:29,2,3,DET 32,0,0,Jared Goff pass incomplete short right,1.010,0.300,DET,32,14.483333
3,1,14:25,3,3,DET 32,0,0,Jared Goff pass incomplete deep right intended...,0.300,-1.240,DET,32,14.416667
4,1,14:19,4,3,DET 32,0,0,"Jack Fox punts 61 yards, returned by Richie Ja...",-1.240,0.320,DET,32,14.316667
...,...,...,...,...,...,...,...,...,...,...,...,...,...
127,4,1:53,2,5,KAN 25,21,20,David Montgomery right guard for 3 yards (tack...,3.710,3.400,KAN,25,1.883333
128,4,1:47,3,2,KAN 22,21,20,David Montgomery up the middle for 2 yards (ta...,3.400,4.240,KAN,22,1.783333
129,4,1:42,1,10,KAN 20,21,20,Jared Goff kneels for -1 yards,4.240,3.560,KAN,20,1.700000
130,4,1:05,2,11,KAN 21,21,20,Jared Goff kneels for -1 yards,3.560,2.740,KAN,21,1.083333
